In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [5]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached target. Stopped...")
      self.model.stop_training=True

callbacks = myCallback()

--2020-09-09 04:06:17--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 173.194.76.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-09-09 04:06:17 (138 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [6]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
# Your Code Here
# This is the first convolution
tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
tf.keras.layers.MaxPooling2D(2, 2),
# The second convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# The third convolution
tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# Flatten the results to feed into a DNN
tf.keras.layers.Flatten(),
# 512 neuron hidden layer
tf.keras.layers.Dense(512, activation='relu'),
# Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [7]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
                "/tmp/h-or-s",  # This is the source directory for training images
                target_size=(150, 150),  # All images will be resized to 150x150
                batch_size=80,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [8]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit_generator(
                train_generator,
                steps_per_epoch=1,  
                epochs=25,
                verbose=1,
                callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/25
1/1 [==============================] - 0s 2ms/step - loss: 0.6893 - accuracy: 0.5000
Epoch 2/25
1/1 [==============================] - 0s 1ms/step - loss: 16.8055 - accuracy: 0.5000
Epoch 3/25
1/1 [==============================] - 0s 1ms/step - loss: 1.8434 - accuracy: 0.5000
Epoch 4/25
1/1 [==============================] - 0s 2ms/step - loss: 0.6895 - accuracy: 0.5000
Epoch 5/25
1/1 [==============================] - 0s 1ms/step - loss: 0.6800 - accuracy: 0.5000
Epoch 6/25
1/1 [==============================] - 0s 1ms/step - loss: 0.6626 - accuracy: 0.5000
Epoch 7/25
1/1 [==============================] - 0s 1ms/step - loss: 0.6105 - accuracy: 0.5000
Epoch 8/25
1/1 [==============================] - 0s 2ms/step - loss: 0.5418 - accuracy: 0.9375
Epoch 9/25
1/1 [==============================] - 0s 1ms/step - loss: 0.4573 - accuracy: 0.7500
Epoch 10/25
1/1 [==============================] - 0s 2ms/step - loss: 0.9337 - accuracy: 0.5000
Epoch 11/25
1/1 [=====================